In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from sklearn.utils import shuffle
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, roc_auc_score, roc_curve, auc, precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from Bio import SeqIO
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, Dropout, MaxPooling1D, Flatten, Dense
import os
import random

In [2]:
def plot(history):
    # learning curves of model accuracy
    plt.plot(history.history['accuracy'], label='train_acc')
    plt.plot(history.history['val_accuracy'], label='val_acc')
    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.legend()
    plt.show()

In [3]:
from sklearn.metrics import accuracy_score, matthews_corrcoef, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
def evaluate_model(model, X_val = None, X_val_pt5 = None, y_val=None):
    y_true = y_val
    # Predict probabilities (or logits if using `from_logits=True`).
    if X_val_pt5 is None:
        y_pred_probs = model.predict(X_val)
    elif X_val is None:
        y_pred_probs = model.predict(X_val_pt5)
    else:
        y_pred_probs = model.predict([X_val, X_val_pt5])

    # Convert probabilities/logits to binary predictions (threshold = 0.5).
    y_pred = (y_pred_probs > 0.5).astype(int)

    # If y_true is one-hot encoded, convert it to binary format
    if len(y_true.shape) > 1 and y_true.shape[1] > 1:  # Check if y_true is one-hot encoded
        y_true = np.argmax(y_true, axis=1)  # Convert one-hot encoded y_true to binary labels

    # Ensure y_pred is also 1D
    if len(y_pred.shape) > 1 and y_pred.shape[1] > 1:
        y_pred = np.argmax(y_pred, axis=1)  # Convert y_pred to binary labels if necessary

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    auprc = average_precision_score(y_true, y_pred_probs)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    # Compute Specificity
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)

    # Print the results
    print(f'Accuracy: {accuracy}')
    print(f'MCC: {mcc}')
    print(f'AUC: {auc}')
    print(f'AUPRC: {auprc}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'Specificity: {specificity}')
    print(f'F1: {f1}')

    return accuracy, mcc, auc, auprc, precision, recall, specificity, f1

In [4]:
from keras.layers import Input, Embedding, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy

def create_conv_branch(input_shape_conv):
    conv_input = Input(shape=input_shape_conv, name='conv_input')

    # Embedding layer
    x = Embedding(input_dim=256, output_dim=21, input_length=input_shape_conv[0])(conv_input)

    x = Lambda(lambda x: tf.expand_dims(x, 3))(x)

    # Convolutional layers
    x = Conv2D(32, kernel_size=(17, 3), activation='relu',
               kernel_initializer='he_normal', padding='VALID')(x)
    x = Dropout(0.2)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)

    x = Dense(16, activation='relu', kernel_initializer='he_normal')(x)
    x = Dropout(0.2)(x)

    # Output of convolutional branch
    conv_output = Dense(16, activation='relu', name='conv_output')(x)

    conv_output = Dense(1, activation='sigmoid')(conv_output)

    model = Model(inputs=conv_input, outputs=conv_output, name='conv_branch')

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss=BinaryCrossentropy(),
                  metrics=['accuracy'])

    return model
# # Instantiate the convolutional branch
# conv_branch = create_conv_branch((33,))

# # Train the convolutional branch
# conv_history = conv_branch.fit(
#     X_train, y_train,
#     epochs=100,
#     batch_size=256,
#     verbose=1,
#     validation_data=(X_val, y_val),
#     callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
# )

# # Optionally, save the trained weights
# conv_branch.save_weights('conv_branch.weights.h5')


In [5]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model

def create_ann_branch(input_shape_ann):
    ann_input = Input(shape=(input_shape_ann,), name='ann_input')

    x = Dense(256, activation='relu')(ann_input)
    x = Dropout(0.4)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.4)(x)

    # Output of ANN branch
    ann_output = Dense(1, activation='sigmoid', name='ann_output')(x)

    model = Model(inputs=ann_input, outputs=ann_output, name='ann_branch')

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss=BinaryCrossentropy(),
                  metrics=['accuracy'])

    return model

# # Instantiate the ANN branch
# ann_branch = create_ann_branch(1024)

# # Train the ANN branch
# ann_history = ann_branch.fit(
#     X_train_pt5, y_train,
#     epochs=100,
#     batch_size=256,
#     verbose=1,
#     validation_data=(X_val_pt5, y_val),
#     callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
# )

# # Optionally, save the trained weights
# ann_branch.save_weights('ann_branch.weights.h5')


In [6]:
from keras.layers import Concatenate, Dense
from keras.models import Model

def create_combined_model(conv_branch, ann_branch):
    # Freeze the branches if you don't want to train them initially
    conv_branch.trainable = False
    ann_branch.trainable = False

    # Define inputs
    conv_input = conv_branch.input
    ann_input = ann_branch.input

    # Get outputs from the branches
    conv_output = conv_branch.get_layer(index=6).output
    ann_output = ann_branch.get_layer(index=4).output

    # Concatenate the outputs
    combined = Concatenate()([conv_output, ann_output])

    # Add combined layers
    x = Dense(16, activation='relu')(combined)
    x = Dense(4, activation='relu')(x)
    output = Dense(1, activation='sigmoid', name='output')(x)

    # Define the combined model
    combined_model = Model(inputs=[conv_input, ann_input], outputs=output, name='combined_model')

    # Compile the combined model
    combined_model.compile(optimizer=Adam(learning_rate=0.001),
                           loss=BinaryCrossentropy(),
                           metrics=['accuracy'])

    return combined_model

# # If you saved the weights separately, load them
# conv_branch.load_weights('conv_branch.weights.h5')
# ann_branch.load_weights('ann_branch.weights.h5')

# # Create the combined model
# combined_model = create_combined_model(conv_branch, ann_branch)

# # View the summary
# combined_model.summary()

# # Train the combined model
# combined_history = combined_model.fit(
#     [X_train, X_train_pt5], y_train,
#     epochs=100,
#     batch_size=256,
#     verbose=1,
#     validation_data=([X_val, X_val_pt5], y_val),
#     callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
# )


# # Evaluate the model
# evaluate_model(combined_model, X_val, X_val_pt5, y_val)
# evaluate_model(combined_model, X_test, X_test_pt5, y_test)




In [7]:
def set_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

#read train and test datasets

train = pd.read_csv('../Embeddings/Prot_t5/train_t5.csv')
val = pd.read_csv('../Embeddings/Prot_t5/val_t5.csv')
test = pd.read_csv('../Embeddings/Prot_t5/test_t5.csv')

print(train.shape)
print(val.shape)
print(test.shape)

# Convert the embedding strings to numpy arrays
X_train_embeddings = train['embedding'].apply(lambda x: np.array([float(i) for i in x.strip('[]').split()]))
X_val_embeddings = val['embedding'].apply(lambda x: np.array([float(i) for i in x.strip('[]').split()]))
X_test_embeddings = test['embedding'].apply(lambda x: np.array([float(i) for i in x.strip('[]').split()]))

# Convert to a numpy array if needed
X_train_embeddings = np.stack(X_train_embeddings.values)
X_val_embeddings = np.stack(X_val_embeddings.values)
X_test_embeddings = np.stack(X_test_embeddings.values)

# Extract sequences
X_train = train['sequence'].values
X_val = val['sequence'].values
X_test = test['sequence'].values

# Extract labels
y_train = train['label'].values
y_val = val['label'].values
y_test = test['label'].values

# Create a dictionary to map amino acids to integers
amino_acids_perm = [
''.join(np.random.permutation(list('ACDEFGHIKLMNPQRSTVWY-'))) for _ in range(20)
]

accuracys = []
mccs = []
aucs = []
auprcs = []
precisions = []
recalls = []
specificitys = []
f1s = []

for amino_acids in amino_acids_perm:
    aa_to_int = {aa: i for i, aa in enumerate(amino_acids)}

    # Convert the sequences to a numerical format and convert to numpy arrays
    X_train_num = [[aa_to_int[aa] for aa in seq] for seq in X_train]
    X_val_num = [[aa_to_int[aa] for aa in seq] for seq in X_val]
    X_test_num = [[aa_to_int[aa] for aa in seq] for seq in X_test]

    X_train_num = np.array(X_train_num)
    X_val_num = np.array(X_val_num)
    X_test_num = np.array(X_test_num)

    set_seed(4)

    # Instantiate the convolutional branch
    conv_branch = create_conv_branch((33,))

    # Train the convolutional branch
    conv_history = conv_branch.fit(
        X_train_num, y_train,
        epochs=100,
        batch_size=256,
        verbose=0,
        validation_data=(X_val_num, y_val),
        callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
    )

    # # Instantiate the ANN branch
    ann_branch = create_ann_branch(1024)

    # Train the ANN branch
    ann_history = ann_branch.fit(
        X_train_embeddings, y_train,
        epochs=100,
        batch_size=256,
        verbose=0,
        validation_data=(X_val_embeddings, y_val),
        callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
    )

    # Create the combined model
    combined_model = create_combined_model(conv_branch, ann_branch)

    # View the summary
    # combined_model.summary()

    # Train the combined model
    combined_history = combined_model.fit(
        [X_train_num, X_train_embeddings], y_train,
        epochs=100,
        batch_size=256,
        verbose=1,
        validation_data=([X_val_num, X_val_embeddings], y_val),
        callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
    )

    # Evaluate the model
    acc, mcc, auc, auprc, precision, recall, specificity, f1 = evaluate_model(combined_model, X_val=X_val_num, X_val_pt5=X_val_embeddings, y_val=y_val)

    evaluate_model(combined_model, X_val=X_test_num, X_val_pt5=X_test_embeddings, y_val=y_test)

    accuracys.append(acc)
    mccs.append(mcc)
    aucs.append(auc)
    auprcs.append(auprc)
    precisions.append(precision)
    recalls.append(recall)
    specificitys.append(specificity)
    f1s.append(f1)

# Print the results mean and std
print(f'Accuracy: {np.mean(accuracys)} +/- {np.std(accuracys)}')
print(f'MCC: {np.mean(mccs)} +/- {np.std(mccs)}')
print(f'AUC: {np.mean(aucs)} +/- {np.std(aucs)}')
print(f'AUPRC: {np.mean(auprcs)} +/- {np.std(auprcs)}')
print(f'Precision: {np.mean(precisions)} +/- {np.std(precisions)}')
print(f'Recall: {np.mean(recalls)} +/- {np.std(recalls)}')
print(f'Specificity: {np.mean(specificitys)} +/- {np.std(specificitys)}')
print(f'F1: {np.mean(f1s)} +/- {np.std(f1s)}')

(8411, 5)
(935, 5)
(3226, 5)


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Epoch 1/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.6091 - loss: 0.6859 - val_accuracy: 0.7412 - val_loss: 0.6523
Epoch 2/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7592 - loss: 0.6222 - val_accuracy: 0.7626 - val_loss: 0.5579
Epoch 3/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7841 - loss: 0.5136 - val_accuracy: 0.7765 - val_loss: 0.4937
Epoch 4/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8017 - loss: 0.4450 - val_accuracy: 0.7786 - val_loss: 0.4796
Epoch 5/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8138 - loss: 0.4132 - val_accuracy: 0.7754 - val_loss: 0.4793
Epoch 6/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8202 - loss: 0.4000 - val_accuracy: 0.7743 - val_loss: 0.4812
Epoch 7/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8156 - loss: 0.3936 - val_accuracy: 0.7722 - val_loss: 0.4840
Epoch 8/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8220 - loss: 0.3870 - val_accuracy: 0

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.6616 - loss: 0.6849 - val_accuracy: 0.7561 - val_loss: 0.6492
Epoch 2/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7784 - loss: 0.6172 - val_accuracy: 0.7701 - val_loss: 0.5518
Epoch 3/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7898 - loss: 0.5046 - val_accuracy: 0.7701 - val_loss: 0.4916
Epoch 4/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7992 - loss: 0.4419 - val_accuracy: 0.7701 - val_loss: 0.4844
Epoch 5/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8087 - loss: 0.4154 - val_accuracy: 0.7743 - val_loss: 0.4876
Epoch 6/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8149 - loss: 0.4047 - val_accuracy: 0.7711 - val_loss: 0.4889
Epoch 7/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8142 - loss: 0.3992 - val_accuracy: 0.7711 - val_loss: 0.4881
Epoch 8/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8243 - loss: 0.3934 - val_accuracy: 0.

...

Epoch 1/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.6224 - loss: 0.6861 - val_accuracy: 0.7668 - val_loss: 0.6552
Epoch 2/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7838 - loss: 0.6250 - val_accuracy: 0.7733 - val_loss: 0.5586
Epoch 3/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7941 - loss: 0.5069 - val_accuracy: 0.7775 - val_loss: 0.4896
Epoch 4/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8039 - loss: 0.4368 - val_accuracy: 0.7765 - val_loss: 0.4805
Epoch 5/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8103 - loss: 0.4128 - val_accuracy: 0.7765 - val_loss: 0.4820
Epoch 6/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8178 - loss: 0.4032 - val_accuracy: 0.7786 - val_loss: 0.4832
Epoch 7/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8157 - loss: 0.3998 - val_accuracy: 0.7807 - val_loss: 0.4816
Epoch 8/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8195 - loss: 0.3973 - val_accuracy: 0.

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.6660 - loss: 0.6862 - val_accuracy: 0.7615 - val_loss: 0.6523
Epoch 2/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7814 - loss: 0.6206 - val_accuracy: 0.7701 - val_loss: 0.5556
Epoch 3/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7860 - loss: 0.5070 - val_accuracy: 0.7754 - val_loss: 0.4953
Epoch 4/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7985 - loss: 0.4467 - val_accuracy: 0.7722 - val_loss: 0.4899
Epoch 5/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8040 - loss: 0.4251 - val_accuracy: 0.7711 - val_loss: 0.4918
Epoch 6/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8065 - loss: 0.4166 - val_accuracy: 0.7775 - val_loss: 0.4906
Epoch 7/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8058 - loss: 0.4145 - val_accuracy: 0.7807 - val_loss: 0.4863
Epoch 8/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8095 - loss: 0.4078 - val_accuracy: 0.

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.6501 - loss: 0.6373 - val_accuracy: 0.7711 - val_loss: 0.5439
Epoch 2/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8034 - loss: 0.4924 - val_accuracy: 0.7797 - val_loss: 0.4864
Epoch 3/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8151 - loss: 0.4126 - val_accuracy: 0.7775 - val_loss: 0.4768
Epoch 4/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8249 - loss: 0.3929 - val_accuracy: 0.7829 - val_loss: 0.4690
Epoch 5/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8295 - loss: 0.3777 - val_accuracy: 0.7893 - val_loss: 0.4644
Epoch 6/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8422 - loss: 0.3651 - val_accuracy: 0.7904 - val_loss: 0.4633
Epoch 7/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8403 - loss: 0.3604 - val_accuracy: 0.7925 - val_loss: 0.4596
Epoch 8/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8453 - loss: 0.3512 - val_accuracy: 0.